# Selecting an LMS

A learning management system (LMS) is a software application for the administration, documentation, tracking, reporting, automation, and delivery of educational courses, training programs, materials or learning and development programs.

There are hundreds of vendors that cropped up in searching for platforms and features. For examples, eLearning Industry list 926 LMS listings. However, not all listings are created equal. The following parameters were used when considering the starting list of potential technologies:

- Software as a Service (SaaS): A licensing model in which the software is licensed on a subscription basis and is centrally hosted. By selecting SaaS only tools, we exclude self-hosted tools.
- Market: We focused only on tools aimed at the non-profit sector.

In [277]:
# Load required libraries:
import requests
import pandas as pd
import altair as alt
import json
from itertools import cycle

## Take a look at the market

Pull from elearningindustry.com the platforms that service the narrow parameters we are looking for. Then store that data for future use:

In [295]:
# Declare search
url = "https://elearningindustry.com/content/directory/listings?category=7113&sort=rating&filters[7107_deployment][]=158241&filters[7107_market][]=158340&offset=0&limit=1000"
res = requests.get(url)
# Save for later
with open('filtered.json', 'w') as f:
    json.dump(res.json(), f)


Open and process the pulled data:

In [311]:
with open('filtered.json') as f:
    data = json.load(f)

def process_response(url_json: list) -> dict:
    """ Process HTTP Response
    Flatten nested HTTP request into a simpler document.

    Parameters
    ----------
    url_json: list
        HTTP response json produced using the .json() method from a Response object from the requests library. 
    """
    out = {}
    for dict in url_json:
        out[dict['title']] = {}
        out[dict['title']]['name'] = dict['title']
        out[dict['title']]['url'] = dict['website']
        out[dict['title']]['rating'] = dict['rating']
        out[dict['title']]['review'] = dict['reviewCount']
        out[dict['title']]['rank'] = dict['rank']

        for deployment in dict['deployment']:
            out[dict['title']]['deployment - {dep}'.format(dep = deployment['name'])] = True
        for integration in dict['integrations']:
            out[dict['title']]['integrations - {int}'.format(int = integration['name'])] = True
        for feature in dict['features']:
            out[dict['title']]['features - {cat} - {feat}'.format(cat = feature['parent'], feat = feature['name'])] = True
    return(out)

def process_dataframe(df: pd.DataFrame, only_true: bool = True) -> pd.DataFrame:
    """ Process pd.DataFrame from Response

    Ingest a messy DataFrame and clean it up!
    """
    for column in df.columns:
        if ' - ' in column:
            df[column] = df[column].fillna(False)
    wanted_columns = [col for col in df.columns if ' - ' in col]
    df = df.melt(id_vars=list(set(df.columns) - set(wanted_columns)),
                 value_vars=wanted_columns,
                 var_name='Quality', value_name='Bool')
    df['Type'] = df['Quality'].str.split(' - ').str[0]
    df['Feat_Cat'] = df['Quality'].str.extract(r'- (.*?) -')
    df['Quality'] = df['Quality'].str.split(' - ').str[-1]
    

    if only_true:
        df = df[df['Bool'] == True]

    return(df)

# Extract and process data as a DataFrame
df = process_dataframe(
        pd.DataFrame.from_dict(
            process_response(data),
            orient='index')
            )

Reshape the data as needed:

In [312]:
def get_features_axis_order(url_json: list) -> dict:
    """ Extract Feateures for Graphing

    Parameters
    ----------
    url_json: list
        HTTP response json produced using the .json() method from a Response object from the requests library. 
    """
    out = {}
    for dict in url_json:
        for feature in dict['features']:
            if feature['parent'] in out.keys():
                if feature['name'] not in out[feature['parent']]:
                    out[feature['parent']].append(feature['name'])
                else:
                    pass
            else:
                out[feature['parent']] = []
                out[feature['parent']].append(feature['name'])
    for key, value in out.items():
        out[key] = sorted(value)

    return(out)

order_list = []
for sub in get_features_axis_order(data).values():
    order_list.extend(sub)

category_list = []
for sub in get_features_axis_order(data).keys():
    rep = len(get_features_axis_order(data)[sub])
    category_list.extend([sub] * rep)

char_colors = {}
colors = ['red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink', 'cyan', 'magenta', 'brown', 'gray', 'teal', 'navy', 'olive', 'lime', 'maroon', 'aqua', 'silver', 'gold', 'black']
color_cycle = cycle(colors)

color_list = []
for char in category_list:
    if char not in char_colors:
        char_colors[char] = next(color_cycle)
    color_list.append(char_colors[char])

category_mapping = {
    '158237': 'red',
    '158282': 'blue',
    '158318': 'green',
    '158438': 'yellow',
    '158483': 'orange',
    '158552': 'purple',
    '158624': 'pink',
    '158732': 'cyan',
    '158765': 'magenta',
    '158834': 'brown',
    '158879': 'grey',
    '158942': 'teal',
    '159005': 'navy',
    '159044': 'olive',
    '159101': 'lime',
    '159200': 'maroon',
    '159242': 'aqua',
    '159275': 'silver',
    '159308': 'gold',
    '159377': 'black'
}

df['f_colors'] = df['Feat_Cat'].replace(category_mapping)

top10 = df.groupby('name')['review'].max().sort_values(ascending=False)[0:10]
counts = df[(df['Type'] == 'features') & (df['name'].isin(top10.index))].groupby('Quality').size().reset_index(name='qcount')
df = pd.merge(df, counts, on='Quality', how='left')

Graph the data:

In [314]:
alt.data_transformers.disable_max_rows()
chart = alt.Chart(data = df[(df['Type'] == 'features') & (df['name'].isin(top10.index))]
          ).mark_rect().encode(
    alt.X('Quality', sort=order_list).title("").axis(labelAngle=90),
    alt.Y('name', sort=list(top10.index)).title(""),
    alt.Color('f_colors:N', legend=None, scale=None)
    ).properties(
    title={
        'text':'Top 10 SaaS LMS for Non Profits',
        'subtitle':'Features colored by class and ranked by  \n',
        'subtitleColor': 'grey'
    }
)

text = alt.Chart(data = (df[(df['Type'] == 'features') & (df['name']
                                                        .isin(top10.index))]
                        .groupby(['Quality'])
                        .first()
                        .reset_index())
                 ).mark_text(dy=-5).encode(
                     x=alt.X('Quality', sort=order_list),
                     y=alt.value(0),
                     text=alt.Text('qcount')
                 )

out  = chart + text

out.configure_view(
    step=13,
    strokeWidth=0
).configure_title(
    anchor='start',
).configure_scale(
    bandPaddingInner=0.1)

alt.LayerChart(...)

Take aways:
- Most have similar features; however, the relevant ones to our project are:
    - Accounts: Add new users, user profiles
    - Activity grading: Course history
    - Authentication: Custom User login page, no login, self-registration with admin confirmation
    - Certificate management: unique certificate by course, predetermined certificate templates, manage certification templates, certification life-cycle
    - Course creation: Assignments, live events, survey engine
    - Enrollment: Attendance tracking, manual enrollment, self-enrollment
    - Format: Course discussions, learner uploads, live chat, social format, topics format, VTC conferencing
    - Learning types: Asynchronous instructor-led, asynchronous self-paced, blended learning, synchronous virtual classroom
    - Interface options: Leaning accessibility, ready-made themes
- Only 350 Learning is listed as capable of supporting "learning accessibility".

Costs:
| Platform | Cost | Description | URL |
|----------|------|-------------|-----|
| TalentLMS | $149/month | 100 users, unlimited courses |  https://www.talentlms.com/prices |
| 360Learning | $8 per active user up to 100 users | NA | https://360learning.com/pricing/ |
| Moodle | NA | NA | NA | 



## Comparison Against All Listings

Remove the industry specific tools an rerun the 

In [266]:
url = "https://elearningindustry.com/content/directory/listings?category=7113&sort=rating&filters[7107_deployment][]=158241&offset=0&limit=1000"
res = requests.get(url)
with open('nofilter.json', 'w') as f:
    json.dump(res.json(), f)

In [275]:
with open('nofilter.json') as f:
    data = json.load(f)
    
# Extract and process data as a DataFrame
nf_data = process_dataframe(
        pd.DataFrame.from_dict(
            process_response(data),
            orient='index')
            )

order_list = []
for sub in get_features_axis_order(data).values():
    order_list.extend(sub)

category_list = []
for sub in get_features_axis_order(data).keys():
    rep = len(get_features_axis_order(data)[sub])
    category_list.extend([sub] * rep)

char_colors = {}
colors = ['red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink', 'cyan', 'magenta', 'brown', 'gray', 'teal', 'navy', 'olive', 'lime', 'maroon', 'aqua', 'silver', 'gold', 'black']
color_cycle = cycle(colors)

color_list = []
for char in category_list:
    if char not in char_colors:
        char_colors[char] = next(color_cycle)
    color_list.append(char_colors[char])

category_mapping = {
    '158237': 'red',
    '158282': 'blue',
    '158318': 'green',
    '158438': 'yellow',
    '158483': 'orange',
    '158552': 'purple',
    '158624': 'pink',
    '158732': 'cyan',
    '158765': 'magenta',
    '158834': 'brown',
    '158879': 'grey',
    '158942': 'teal',
    '159005': 'navy',
    '159044': 'olive',
    '159101': 'lime',
    '159200': 'maroon',
    '159242': 'aqua',
    '159275': 'silver',
    '159308': 'gold',
    '159377': 'black'
}

nf_data['f_colors'] = nf_data['Feat_Cat'].replace(category_mapping)

top20 = nf_data.groupby('name')['review'].max().sort_values(ascending=False)[0:20]
counts = nf_data[(nf_data['Type'] == 'features') & (nf_data['name'].isin(top20.index))].groupby('Quality').size().reset_index(name='qcount')
nf_data = pd.merge(nf_data, counts, on='Quality', how='left')

In [276]:
alt.data_transformers.disable_max_rows()
chart = alt.Chart(data = nf_data[(nf_data['Type'] == 'features') & (nf_data['name'].isin(top20.index))]
          ).mark_rect().encode(
    alt.X('Quality', sort=order_list).title("").axis(labelAngle=90),
    alt.Y('name', sort=list(top20.index)).title(""),
    alt.Color('f_colors:N', legend=None, scale=None)
    ).properties(
    title={
        'text':'Top 20 SaaS LMS',
        'subtitle':'Features colored by class \n',
        'subtitleColor': 'grey'
    }
)

text = alt.Chart(data = (df[(df['Type'] == 'features') & (df['name']
                                                        .isin(top20.index))]
                        .groupby(['Quality'])
                        .first()
                        .reset_index())
                 ).mark_text(dy=-5).encode(
                     x=alt.X('Quality', sort=order_list),
                     y=alt.value(0),
                     text=alt.Text('qcount')
                 )

out  = chart + text

out.configure_view(
    step=13,
    strokeWidth=0
).configure_title(
    anchor='start',
).configure_scale(
    bandPaddingInner=0.1)

alt.LayerChart(...)